In [1]:
from qdrant_client import QdrantClient
from qdrant_client.http import models
import numpy as np
from faker import Faker

In [2]:
client = QdrantClient(host="localhost",port=6333)
client

In [4]:
my_collection = "first_collection"
client.create_collection(
    collection_name = my_collection,
    vectors_config= models.VectorParams(size = 100,distance=models.Distance.COSINE)
)

True

In [5]:
data = np.random.uniform(low=-1.0,high=1.0,size=(1_000,100))
index = list(range(1_000))

In [6]:
data.shape

(1000, 100)

In [7]:
data[:2,:10]

array([[-0.26243889,  0.37837476,  0.78066389,  0.81543478,  0.12163599,
         0.98527251,  0.91013734,  0.48162548, -0.4784776 ,  0.24584863],
       [ 0.0705615 , -0.32073983,  0.41972394, -0.20733712,  0.09579856,
         0.42461256,  0.04712729, -0.43496611,  0.69363004, -0.04421058]])

In [8]:
index[-10:]

[990, 991, 992, 993, 994, 995, 996, 997, 998, 999]

In [9]:
client.upsert(
    collection_name = my_collection,
    points = models.Batch(
        ids= index,
        vectors=data.tolist()
    )
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [10]:
client.retrieve(
    collection_name=my_collection,
    ids = [10,14,100],
    #with_vectors=True
)

[Record(id=10, payload={}, vector=None, shard_key=None, order_value=None),
 Record(id=14, payload={}, vector=None, shard_key=None, order_value=None),
 Record(id=100, payload={}, vector=None, shard_key=None, order_value=None)]

In [11]:
fake_something = Faker()
fake_something.name() , fake_something.address()

('Charles Hodge', '5342 Madison Alley Suite 069\nSwansonville, AL 85302')

In [12]:
payload = []
for i in range(1000):
    payload.append(
        {
            "artist":fake_something.name(),
            "song" : " ".join(fake_something.words()),
            "url_Song" : fake_something.url(),
            "year": fake_something.year(),
            "country" : fake_something.country()
        }
    )

In [13]:
print(payload[1])

{'artist': 'Nicole Hill', 'song': 'moment national owner', 'url_Song': 'http://bryant-lamb.biz/', 'year': '1987', 'country': 'Gabon'}


In [14]:
client.upsert(
    collection_name = my_collection,
    points = models.Batch(
        ids = index,
        vectors = data.tolist(),
        payloads=payload
    )
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

In [15]:
living_la_vida_loca = np.random.uniform(low=-1.0,high=1.0,size=(100)).tolist()
living_la_vida_loca[:5]

[0.9040839916828014,
 0.7540234434154545,
 0.26258669514435407,
 0.7747655367528976,
 0.2386053982083729]

In [16]:
client.search(
    collection_name = my_collection,
    query_vector = living_la_vida_loca,
    limit=5
)

[ScoredPoint(id=672, version=1, score=0.3094275, payload={'artist': 'Tyler Ortiz', 'country': 'Kazakhstan', 'song': 'suddenly somebody add', 'url_Song': 'http://www.smith-rodriguez.com/', 'year': '2001'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=985, version=1, score=0.3003493, payload={'artist': 'Jennifer Barker', 'country': 'French Southern Territories', 'song': 'present might recognize', 'url_Song': 'http://www.briggs-ali.com/', 'year': '2002'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=412, version=1, score=0.26847097, payload={'artist': 'Mary Evans', 'country': 'El Salvador', 'song': 'interview grow side', 'url_Song': 'https://www.jones.org/', 'year': '2005'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=909, version=1, score=0.26511016, payload={'artist': 'Jeremy Price', 'country': 'Switzerland', 'song': 'environment born agency', 'url_Song': 'https://www.jones.com/', 'year': '1986'}, vector=None, shard_key=None, ord

In [17]:
aussie_songs= models.Filter(
    must = [
        models.FieldCondition(
            key="country",match = models.MatchValue(value="Taiwan")
        )
    ]
)
aussie_songs

Filter(should=None, min_should=None, must=[FieldCondition(key='country', match=MatchValue(value='Taiwan'), range=None, geo_bounding_box=None, geo_radius=None, geo_polygon=None, values_count=None)], must_not=None)

In [18]:
client.search(
    collection_name = my_collection,
    query_vector = living_la_vida_loca,
    query_filter=aussie_songs,
    limit=5
)

[ScoredPoint(id=484, version=1, score=0.13764292, payload={'artist': 'Dawn Benton', 'country': 'Taiwan', 'song': 'herself themselves sit', 'url_Song': 'http://www.bruce.com/', 'year': '1993'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=360, version=1, score=0.0021311473, payload={'artist': 'Matthew Graves', 'country': 'Taiwan', 'song': 'yourself example thousand', 'url_Song': 'http://sloan.com/', 'year': '1984'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=628, version=1, score=-0.027391953, payload={'artist': 'Dr. Steven Andrade', 'country': 'Taiwan', 'song': 'these relationship present', 'url_Song': 'https://www.cervantes.com/', 'year': '2002'}, vector=None, shard_key=None, order_value=None)]

In [19]:
client.recommend(
    collection_name = my_collection,
    #query_vector = living_la_vida_loca,
    positive = [17],
    negative = [100,444],
    limit = 5
)

[ScoredPoint(id=455, version=1, score=0.3103735, payload={'artist': 'Christopher Gonzalez', 'country': 'Eritrea', 'song': 'face talk also', 'url_Song': 'https://phillips.biz/', 'year': '1977'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=6, version=1, score=0.2744976, payload={'artist': 'Steven Nguyen', 'country': 'New Caledonia', 'song': 'yet blood generation', 'url_Song': 'http://www.thomas-martin.com/', 'year': '2013'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=298, version=1, score=0.27153775, payload={'artist': 'Michael Underwood', 'country': 'Albania', 'song': 'including his guess', 'url_Song': 'http://buck.com/', 'year': '1983'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=768, version=1, score=0.26983914, payload={'artist': 'Kent Brown', 'country': 'Saint Martin', 'song': 'ago it five', 'url_Song': 'http://www.roth-kennedy.biz/', 'year': '1970'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=220, ve